In [1]:
import numpy as np


#加載數據集
def load_data():
    with open('連續值數據.txt') as fr:
        lines = fr.readlines()

    x = np.empty((len(lines), 9), dtype=float)

    for i in range(len(lines)):
        line = lines[i].strip().split(',')
        x[i] = line

    test_x = x[10:]
    x = x[:10]

    return x, test_x


x, test_x = load_data()
x, test_x

(array([[0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.697, 0.46 , 0.   ],
        [1.   , 0.   , 1.   , 0.   , 0.   , 0.   , 0.774, 0.376, 0.   ],
        [1.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.634, 0.264, 0.   ],
        [0.   , 1.   , 0.   , 0.   , 1.   , 1.   , 0.403, 0.237, 0.   ],
        [1.   , 1.   , 0.   , 1.   , 1.   , 1.   , 0.481, 0.149, 0.   ],
        [0.   , 2.   , 2.   , 0.   , 2.   , 1.   , 0.243, 0.267, 1.   ],
        [2.   , 1.   , 1.   , 1.   , 0.   , 0.   , 0.657, 0.198, 1.   ],
        [1.   , 1.   , 0.   , 0.   , 1.   , 1.   , 0.36 , 0.37 , 1.   ],
        [2.   , 0.   , 0.   , 2.   , 2.   , 0.   , 0.593, 0.042, 1.   ],
        [0.   , 0.   , 1.   , 1.   , 1.   , 0.   , 0.719, 0.103, 1.   ]]),
 array([[0.   , 0.   , 1.   , 0.   , 0.   , 0.   , 0.608, 0.318, 0.   ],
        [2.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.556, 0.215, 0.   ],
        [1.   , 1.   , 0.   , 0.   , 1.   , 0.   , 0.437, 0.211, 0.   ],
        [1.   , 1.   , 1.   , 1.   , 1.   , 0.   

計算數據集的熵。熵是表示數據集的不確定性的一種度量，數值越高表示數據集越不確定。

In [2]:
#計算數據集的熵,當然這個熵是針對於y來說的
def get_entropy(_x):
    entropy = 0

    #統計y的熵就可以了
    _y = _x[:, -1]
    _y = _y.astype(np.int)

    #統計每個結果出現的次數,[8 9],表示0出現8次,1出現9次
    bincount = np.bincount(_y)  #[8 9]
    for count in bincount:
        if count == 0:
            continue

        #出現次數 / 總次數 = 出現概率
        prob = count / len(_x)

        #熵 = p * log(p) * -1
        entropy -= prob * np.log2(prob)

    return entropy


#數字越大,混亂度越高
get_entropy(x)

1.0

get_cut_point()函數，用於獲取數據集某一列的切分點。切分點是指將該列數據劃分為兩部分的數值，用於決策樹的劃分。

In [3]:
def get_cut_point(values):
    values = np.unique(values)
    values.sort()
    cut_point = np.empty(len(values) - 1)
    for i in range(len(values) - 1):
        cut_point[i] = (values[i] + values[i + 1]) / 2
    return cut_point


get_cut_point(x[:, 6])

array([0.3015, 0.3815, 0.442 , 0.537 , 0.6135, 0.6455, 0.677 , 0.708 ,
       0.7465])

In [ ]:
get_gain()函數，用於計算某一列的信息增益。信息增益是指某一列數據進行切分後，熵值的下降量。信息增益越大，表示該列數據切分後，對於決策樹的建立具有更大的貢獻。

In [4]:
def get_gain(_x, col):
    #列熵
    min_cut_entropy = 1e20
    min_cut = 0

    #遍歷所有切分點
    for cut in get_cut_point(_x[:, col]):
        cut_entropy = 0

        #切分數據
        for sub_x in [_x[_x[:, col] >= cut], _x[_x[:, col] < cut]]:

            #這個數據子集出現的概率,很顯然,等於出現次數/總次數
            prob = len(sub_x) / len(_x)

            #求數據子集的熵
            entropy = get_entropy(sub_x)

            #這個切分點的熵,等於這個式子的累計
            cut_entropy += prob * entropy

        if cut_entropy < min_cut_entropy:
            min_cut_entropy = cut_entropy
            min_cut = cut

    #信息增益,就是切分數據後,熵值能下降多少,這個值越大越好
    gain = get_entropy(_x) - min_cut_entropy

    return gain, min_cut


get_gain(x, 6)

(0.23645279766002802, 0.3815)

In [ ]:
get_split_col()函數，用於獲取最佳切分列。遍歷所有的列，計算各列的信息增益，選擇信息增益最大的列作為最佳切分列。

In [5]:
def get_split_col(_x):
    best_col = -1
    best_cut = -1
    best_gain = 0

    #遍歷所有的列,最後一列是y,不需要計算
    for col in range(_x.shape[1] - 1):

        #信息增益,就是切分數據後,熵值能下降多少,這個值越大越好
        gain, cut = get_gain(_x, col)

        #信息增益最大的列,就是要被拆分的列
        if gain > best_gain:
            best_gain = gain
            best_col = col
            best_cut = cut

    return best_col, best_cut


get_split_col(x)

(0, 1.5)

Node類和Leaf類，用於構建決策樹節點和葉子對象。其中，Node類包含了屬性col和value，用於記錄節點對應的列和切分點；gt和lt屬性用於記錄節點的子節點；Leaf類包含了屬性y，用於記錄葉子節點對應的分類結果。

In [6]:
#創建節點和葉子對象,用來構建樹
class Node():
    def __init__(self, col, value):
        self.col = col
        self.value = value
        self.gt = None
        self.lt = None

    def __str__(self):
        return 'Node col=%d value=%f' % (self.col, self.value)

    def set_child(self, symbol, node):
        if symbol == 'gt':
            self.gt = node
        if symbol == 'lt':
            self.lt = node


class Leaf():
    def __init__(self, y):
        self.y = y

    def __str__(self):
        return 'Leaf y=%d' % self.y


print(Node(0, 1.0)), print(Leaf(1))

Node col=0 value=1.000000
Leaf y=1


(None, None)

In [7]:
#打印樹的方法
def print_tree(node, prefix='', subfix=''):
    prefix += '-' * 4
    print(prefix, node, subfix)
    if isinstance(node, Leaf):
        return

    if node.gt:
        subfix = 'value=gt'
        print_tree(node.gt, prefix, subfix)
    if node.lt:
        subfix = 'value=lt'
        print_tree(node.lt, prefix, subfix)


print_tree(Node(0, 1.0))

---- Node col=0 value=1.000000 


調用函數get_split_col(x)來獲取最大信息增益的列，並返回該列和其對應的值。

In [8]:
#先在所有數據上求最大信息增益的列,結果是7
col,value = get_split_col(x)
col,value

(0, 1.5)

In [9]:
#根據上面的結果,創建根節點,根節點根據列7的值來分割數據
root = Node(0, 1.5)
print(root)

Node col=0 value=1.500000


定義函數create_children(_x, parent_node)來創建子節點。該函數首先遍歷父節點col列所有的取值，然後根據父節點col列的取值分割數據，如果所有的y都是一樣的，則說明是個葉子節點，否則是個分支節點，需要計算最佳切分列，並添加分支節點到父節點上。

In [10]:
#添加子節點的方法
def create_children(_x, parent_node):

    #遍歷父節點col列所有的取值
    for symbol in ('gt', 'lt'):
        #首先根據父節點col列的取值分割數據
        sub_x = _x[_x[:, parent_node.col] >= parent_node.value]
        if symbol == 'lt':
            sub_x = _x[_x[:, parent_node.col] < parent_node.value]

        #取去重y值
        unique_y = np.unique(sub_x[:, -1])

        #如果所有的y都是一樣的,說明是個葉子節點
        if len(unique_y) == 1:
            parent_node.set_child(symbol, Leaf(unique_y[0]))
            continue

        #否則,是個分支節點,計算最佳切分列
        split_col, value = get_split_col(sub_x)

        #添加分支節點到父節點上
        parent_node.set_child(symbol, Node(col=split_col, value=value))


create_children(x, root)

print_tree(root)

---- Node col=0 value=1.500000 
-------- Leaf y=1 value=gt
-------- Node col=6 value=0.381500 value=lt


In [11]:
#继续创建,0<1.5节点的下一层
x_0_lt_15 = x[x[:, 0] < 1.5]
create_children(x_0_lt_15, root.lt)

print_tree(root)

---- Node col=0 value=1.500000 
-------- Leaf y=1 value=gt
-------- Node col=6 value=0.381500 value=lt
------------ Node col=7 value=0.126000 value=gt
------------ Leaf y=1 value=lt


In [12]:
#继续创建,7>0.126,1>0.5节点的下一层
x_0_lt_15_and_6_gt_03815 = x_0_lt_15[x_0_lt_15[:, 6] >= 0.3815]
create_children(x_0_lt_15_and_6_gt_03815, root.lt.gt)

print_tree(root)

---- Node col=0 value=1.500000 
-------- Leaf y=1 value=gt
-------- Node col=6 value=0.381500 value=lt
------------ Node col=7 value=0.126000 value=gt
---------------- Leaf y=0 value=gt
---------------- Leaf y=1 value=lt
------------ Leaf y=1 value=lt


In [13]:
#预测方法,测试
def pred(_x, node):
    if isinstance(node, Leaf):
        return node.y

    child = node.gt
    if _x[node.col] < node.value:
        child = node.lt

    return pred(_x, child)


correct = 0
for i in x:
    if pred(i, root) == i[-1]:
        correct += 1

print(correct / len(x))

print('-------------------------')

correct = 0
for i in test_x:
    if pred(i, root) == i[-1]:
        correct += 1

print(correct / len(test_x))

1.0
-------------------------
0.7142857142857143
